In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'


In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
!pwd

/home/talentum/test-jupyter/exam


In [8]:
# Reading the file
tips_df = spark.read.csv('file:///home/talentum/test-jupyter/exam/tips.csv', header=True, inferSchema=True)

In [9]:
tips_df.show(4)

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|total_bill| tip|gender|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560330000000000|   Sun2959|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478070000000000|   Sun4608|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011810000000000|   Sun4458|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           11.84|  Nathaniel Harris|4676140000000000|   Sun5260|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
only showing top 4 rows



In [14]:
# Print the schema
tips_df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- price_per_person: double (nullable = true)
 |-- Payer Name: string (nullable = true)
 |-- CC Number: decimal(16,0) (nullable = true)
 |-- Payment ID: string (nullable = true)



In [15]:
# calculating new df with precentage
tips_df_precentage = tips_df.withColumn('tip_percentage', (tips_df.tip/tips_df.total_bill)*100)
tips_df_precentage.show(2)

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+------------------+
|total_bill| tip|gender|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|    tip_percentage|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560330000000000|   Sun2959|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478070000000000|   Sun4608|16.054158607350097|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+------------------+
only showing top 2 rows



In [18]:
tips_above_threshold_df = tips_df_precentage.filter(tips_df_precentage.tip_percentage >= 5)
tips_above_threshold_df.show(5)

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+------------------+
|total_bill| tip|gender|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|    tip_percentage|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560330000000000|   Sun2959|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478070000000000|   Sun4608|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011810000000000|   Sun4458|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           11.84|  Nathaniel Harris|4676140000000000|   Sun5260| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|            6.15|      Tonya Carter|

In [29]:
# Pringting the total tips which re greate than 5$
print(f"Number of tips (Dataframe): {tips_above_threshold_df.count()}")

Number of tips (Dataframe): 243


In [27]:
# Using sql to to the same thing
tips_df.createOrReplaceTempView("tips")


# Writing the quey 
tips_above_threshold_sql = spark.sql("""
    SELECT * 
    FROM tips
    WHERE (tip/ total_bill)*100 >= 5
""")

# Show the result
tips_above_threshold_sql.show(5)

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|total_bill| tip|gender|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560330000000000|   Sun2959|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478070000000000|   Sun4608|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011810000000000|   Sun4458|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           11.84|  Nathaniel Harris|4676140000000000|   Sun5260|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|            6.15|      Tonya Carter|4832730000000000|   Sun2251|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----

In [28]:
# Sohwing total sql results
print(f"Number of tips (SQL): {tips_above_threshold_sql.count()}")

Number of tips (SQL): 243
